# Extraction des données RVL-CDIP

## README
Ce notebook permet de télécharger sur le site de huggingface les images de la BDD RVL-CDIP.

Il réalise tout d'abord certaines opérations préalables (chapitre 1), dont la configuration des répertoires du projet

A l'issue (chapitre 2), il télécharge l'ensemble des documents de la BDD RVL-CDIP puis décompresse les fichiers téléchargés.

Remarque:
- **[IMPORTANT] L'utilisation de ce script écrira environ 50 Go de données (principalement des fichiers tif) sur le disque dur.**.


## 1. Préparation

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import os
import time
import tarfile
import requests
from tqdm import tqdm

## 2. Téléchargement des fichiers RVL-CDIP

### 2.1. Fichiers images

In [ ]:
# Téléchargement de l'archive
url = "https://huggingface.co/datasets/aharley/rvl_cdip/resolve/main/data/rvl-cdip.tar.gz?download=true"
tar_file_path = PATHS.rvl_cdip / "rvl-cdip.tar.gz"

# Stream download with progress bar
response = requests.get(url, stream=True)
total = int(response.headers.get('content-length', 0))
block_size = 1024

with open(tar_file_path, 'wb') as f, tqdm(
    desc="Téléchargement RVL-CDIP",
    total=total,
    unit='iB',
    unit_scale=True,
    unit_divisor=1024
) as bar:
    for data in response.iter_content(block_size):
        f.write(data)
        bar.update(len(data))

In [ ]:
# Extraction des images
with tarfile.open(tar_file_path, "r:gz") as archive:
    archive.extractall(path=PATHS.rvl_cdip)

In [ ]:
# Suppression du fichier archive
image_files = list(PATHS.rvl_cdip_images.rglob("*.tif"))
if len(image_files) == 400000:
    os.remove(tar_file_path)

### 2.2 Autres fichiers

In [ ]:
for label_set in ["train", "val", "test"]:
    url = f"https://huggingface.co/datasets/aharley/rvl_cdip/resolve/main/data/{label_set}.txt?download=true"
    response = requests.get(url, stream=True)
    with open(PATHS.labels / f"{label_set}.txt", "wb") as f:
        f.write(response.content)
        
for additional_file in ["README.md", "dataset_infos.json"]:
    url = f"https://huggingface.co/datasets/aharley/rvl_cdip/resolve/main/{additional_file}?download=true"
    response = requests.get(url, stream=True)
    with open(PATHS.rvl_cdip / additional_file), "wb") as f:
        f.write(response.content)